## 1. Imports

In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## 2. Loading the data

In [30]:
stock_ticker = "GME"

In [31]:
df = pd.read_csv(f"../../../data/stock_prediction/stock_prediction_data/stock_prediction_data_{stock_ticker}.csv", index_col=[0])

In [32]:
# log_return_t+1 is to predict
target = df["log_return_shift"].apply(lambda x: 1 if x > 0 else -1)

# used features to predict stock return
features = ["Negative", "Neutral", "Positive", "count", "pos_minus_neg","pos_minus_neg_times_count", "Volume", "pos_minus_neg_diff", "count_diff"]
df = df[features]


In [33]:
df

,Negative,Neutral,Positive,count,pos_minus_neg,pos_minus_neg_times_count,Volume,pos_minus_neg_diff,count_diff
2021-05-04,0.271763,0.360857,0.367380,499,0.095617,47.712727,4007500,-0.068648,3.0
2021-05-05,0.229515,0.447798,0.322687,820,0.093173,76.401538,2805400,-0.002444,321.0
2021-05-06,0.201018,0.429720,0.369262,1647,0.168243,277.096813,2942800,0.075071,827.0
2021-05-07,0.200771,0.395441,0.403788,734,0.203017,149.014766,2934600,0.034774,-913.0
2021-05-10,0.290007,0.379028,0.330965,756,0.040958,30.964170,4806700,-0.162059,22.0
...,...,...,...,...,...,...,...,...,...
2022-04-22,0.259425,0.387646,0.352929,166,0.093503,15.521564,1569600,0.026664,2.0
2022-04-25,0.251789,0.407347,0.340865,226,0.089076,20.131230,1534800,-0.004427,60.0
2022-04-26,0.341325,0.328281,0.330394,198,-0.010931,-2.164416,1545700,-0.100008,-28.0
2022-04-27,0.228449,0.414548,0.357003,245,0.128553,31.495606,1305900,0.139485,47.0


In [34]:
target.value_counts(normalize=True)

-1    0.532
 1    0.468
Name: log_return_shift, dtype: float64

## 3. Data transformation

In [35]:
# Data normalization
df = (df - df.mean())/df.std()

In [43]:
# Stratified split
X_train, X_test, Y_train, Y_test = train_test_split(df, target, stratify=target, test_size=0.25,random_state=42)

In [49]:
Y_train

2021-10-25    1
2021-05-13   -1
2021-09-15    1
2021-12-10   -1
2021-06-21    1
             ..
2021-12-27   -1
2021-09-29   -1
2022-02-14    1
2021-12-01    1
2022-03-01    1
Name: log_return_shift, Length: 187, dtype: int64

In [48]:
Y_test.value_counts(normalize=True)

-1    0.539683
 1    0.460317
Name: log_return_shift, dtype: float64

## 4. Model training and prediction

In [51]:
# Create LR classifier
clf = LogisticRegression(random_state=0).fit(X_train, Y_train)

In [54]:
clf.score(X_test, Y_test)

0.49206349206349204